In [1]:
import numpy as np
from scipy.stats import wishart

def Gibbs(N,T,x,y,Omega,nchain,init,R,beta,mu0,sigma0):
    chain=np.zeros((nchain+1,3))        # mu_beta1,mu_beta2 et sigma
    latent1=np.zeros((nchain + 1, N))   #beta 1
    latent2=np.zeros((nchain + 1, N))   # beta2
    latent3=[]                          # Omega
    
    latent1[0]=beta[:,0]
    latent2[0]=beta[:,1]
    latent3.append(Omega)
    chain[0]=init
    
    
    for i in range(1,nchain+1):
        beta1=latent1[i-1]  # de taille N
        beta2=latent2[i-1]
        Omega=latent3[i-1]
        mu_beta1=chain[i-1,0]   #mu_beta1
        mu_beta2=chain[i-1,1]   #mu_beta2
        sigma=chain[i-1,2]          # sigma=1/sqrt(tauc)
        
        #update sigma:
        shape_cond=0.001+T*N/2
        mu_col=[beta1+beta2*x[j] for j in range(T)]
        mu=np.column_stack(mu_col)
        rate_cond=0.001+0.5*np.sum((y-mu)**2)
        tauc=np.random.gamma(shape=shape_cond,scale=1/rate_cond)
        sigma=1/np.sqrt(tauc)
        
        #update beta1:
        for k in range(N):
            mean1_cond=(mu_beta1-tauc*Omega[0,0]*(beta2[k]*np.sum(x)-np.sum(y[k])))/(1+tauc*Omega[0,0]*T)
            var1_cond=Omega[0,0]/(1+tauc*Omega[0,0]*T)
            beta1[k]=np.random.normal(mean1_cond,np.sqrt(var1_cond))
            
        #update beta2:
        for k in range(N):
            mean2_cond=(mu_beta2-tauc*Omega[1,1]*(beta1[k]*np.sum(x)-np.sum(y[k])))/(1+tauc*Omega[1,1]*np.sum(x**2))
            var2_cond=Omega[1,1]/(1+tauc*Omega[1,1]*np.sum(x**2))
            beta2[k]=np.random.normal(mean2_cond,np.sqrt(var2_cond))
            
        beta=np.column_stack((beta1, beta2))
        
        #update Omega:
        df=N+2
        mean_beta=np.column_stack((np.full(N, mu_beta1), np.full(N, mu_beta2)))
        S=np.zeros((2,2))
        for k in range(N):
            S=S+np.dot((beta[k]-mean_beta[k]).T,(beta[k]-mean_beta[k]))
        scale=np.linalg.inv(np.linalg.inv(R)+S)
        Omega=wishart.rvs(df, scale)
        
        #update mu_beta1
        mean_mu_beta1=(mu0[0]*Omega[0,0]+sigma0[0,0]**2*np.sum(beta1))/(Omega[0,0]+N*sigma0[0,0]**2)
        var_mu_beta1=sigma0[0,0]**2*Omega[0,0]/(Omega[0,0]+N*sigma0[0,0]**2)
        mu_beta1=np.random.normal(mean_mu_beta1,np.sqrt(var_mu_beta1))
        
        #update mu_beta2
        mean_mu_beta2=(mu0[1]*Omega[1,1]+sigma0[1,1]**2*np.sum(beta2))/(Omega[1,1]+N*sigma0[1,1]**2)
        var_mu_beta2=sigma0[1,1]**2*Omega[1,1]/(Omega[1,1]+N*sigma0[1,1]**2)
        mu_beta2=np.random.normal(mean_mu_beta2,np.sqrt(var_mu_beta2))
        
        ## Store new states
        chain[i]=[mu_beta1,mu_beta2,sigma]
        latent1[i]=beta1
        latent2[i]=beta2
        latent3.append(Omega)
        
    return chain, latent1, latent2,latent3     